# Localize an ArcGIS Hub Site

for hub site
1. create the english/default site
2. use the Hub admin UI to clone the Sites/pages
3. get the Hub Site/Page ID

for groups
- where the group is in the JSON for the Site

## Import Python libraries

In [1]:
from arcgis import GIS
from arcgis.features import FeatureLayer
from arcgis.features import FeatureLayerCollection

import json
import os

## Setup your connection to ArcGIS Online

In [2]:
username = input('Username: ')
gis = GIS('https://www.arcgis.com', username)

gis = GIS('https://www.arcgis.com', username, password)

print ('logged in')

logged in


In [ ]:
unsd_std_fields = None
with open(os.path.abspath('test-files/UNSD_STANDARD_FIELDS.json'), 'rb') as file:
    unsd_std_fields = json.load(file)

print(unsd_std_fields)

In [ ]:
locale_code = 'fr'

In [ ]:
sdg_metadata = None
pth = 'testing/localized/SDG_METADATA_API_{}.json'.format(locale_code)
print (pth)
with open(os.path.abspath(pth), 'rb') as file:
    sdg_metadata = json.load(file)

print(sdg_metadata)

In [ ]:
group_ids = ['66d8595b381440afb5e320a9265c3fe1']

locale_fields = unsd_std_fields[locale_code]

icon_base_url = 'https://github.com/Esri/sdg-dash/raw/master/public/images/sdg/{}/TGG_Icon_Color_0{}.png'

# ago_working_directory = None
ago_working_directory = 'Localization Testing'

for group_id in group_ids:
    group = gis.groups.get(group_id)
    
    items = group.content()
    
    print ('starting in Group :: {}'.format(group.title))
    for i, item in enumerate(items):
        
        # testing :: stop after 2 items
        if i == 2:
            break
            
        # get the feature layer
        fl = item.layers[0]
        
        tags = item.tags
        
        sdg_ind_tag = None
        sdg_series_tag = None
        
        for tag in tags:
            if 'SDGIND' in tag:
                sdg_ind_tag = tag
            elif 'SDGSERIES' in tag:
                sdg_series_tag = tag
        
        if sdg_ind_tag is None or sdg_series_tag is None:
            break
            
        ind_parts = sdg_ind_tag.split('.')
        goal_id = ind_parts[1]
        target_id = '{}.{}'.format(goal_id, ind_parts[2])
        indicator_id = '{}.{}.{}'.format(goal_id, ind_parts[2], ind_parts[3])
        
        series_parts = sdg_series_tag.split('.')
        series_id = series_parts[1]
        print (goal_id,target_id, indicator_id, series_id)

        item_flc = FeatureLayerCollection.fromitem(item)

        temporary_view_name = '{}_{}'.format(tag.replace('.', '_').lower(), locale_code)
        print ('creating hosted fl view :: {} ..'.format(temporary_view_name))
        new_view = item_flc.manager.create_view(name=temporary_view_name)
        
        if ago_working_directory is not None:
            new_view.move(ago_working_directory)

        view_flc = FeatureLayerCollection.fromitem(new_view)
        view_fl = view_flc.layers[0]

        # update the field alias
        fields = view_fl.properties.fields
        for field in fields:
            if field.name in locale_fields:
                field.alias = locale_fields[field.name]

        view_fl.properties.editingInfo.lastEditDate = None

        print ('updating field aliases ..')
        view_fl.manager.update_definition(view_fl.properties)

        # update the AGO Item Card
        view_name = '{} - {}'.format(item.title, locale_code)
        snippet = item.title
        locale_tags = item.tags
        for series in sdg_metadata:
            if series['seriesCode'] == series_id:
                view_name = 'Indicator {} - {}'.format(indicator_id, series['seriesDescription'])
                snippet = view_name
                locale_tags = item.tags + series['TAGS']
                description = """
                    <b>Series {}</b>: {}<br/><br/>
                    <b>Indicator {}</b>: {}<br/><br/>
                    <b>Target {}</b>: {}<br/><br/>
                    <b>Goal {}</b>: {}<br/>
                    """.format(
                    series_id, series['seriesDescription'],
                    indicator_id, series['indicatorDescription'],
                    target_id, series['targetDescription'],
                    goal_id, series['goalDescription'])


        view_item_card = {
            "title": view_name,
            "description": description,
            "snippet": snippet,
            "tags": locale_tags,
            "categories": item.categories
        }

        thumbnail_url = icon_base_url.format(locale_code, goal_id)

        print ('updating Item card ..')
        new_view.update(item_properties=view_item_card, thumbnail=thumbnail_url)

        # share with everyone
        new_view.share(everyone=True)
        
        # share with Open Data Groups
        # ex: Group Name is 'SDG01_fr' for SDG01 french version
        sdg_od_group_locale = locale_groups[goal_id]
        new_view.share(groups=sdg_od_group_locale)
                
print ('done')